# config

> Utilities for loading Stata and nbstata
- order: 1

Before we can use [pystata](https://www.stata.com/python/pystata18/index.html), we need to find the local Stata path (i.e., `find_path`) and then [add pystata to sys.path](https://www.stata.com/python/pystata18/install.html#method-2-adding-pystata-to-sys-path) (i.e., `set_pystata_path`) so it can be imported.

The `get_config` function handles nbstata configuration, more broadly.

In [ ]:
#| default_exp config
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from fastcore.test import test_eq, ExceptionExpected

In [ ]:
#| export
from nbstata.misc_utils import print_red
from fastcore.basics import patch_to
import os
import sys
import platform
from shutil import which
from pathlib import Path
from packaging import version
import configparser

## pystata configuration

### Auto-detect Stata directory and edition

In [ ]:
#| export
def _win_find_path(_dir=None):
    if _dir is None:
        dirs = [r'C:\Program Files\StataNow19',
                r'C:\Program Files\Stata19',
                r'C:\Program Files\StataNow18',
                r'C:\Program Files\Stata18',
                r'C:\Program Files\Stata17']
    else:
        dirs = [_dir]
    for this_dir in dirs:
        path = Path(this_dir)
        if os.path.exists(path):
            executables = [exe for exe in path.glob("Stata*.exe") if exe not in set(path.glob("Stata*_old.exe"))]
            if executables:
                return str(executables[0])
    # Otherwise, try old way
    import winreg
    reg = winreg.ConnectRegistry(None, winreg.HKEY_CLASSES_ROOT)
    subkey = r'Stata17Do\shell\do\command'
    try:
        key = winreg.OpenKey(reg, subkey)
        return winreg.QueryValue(key, None).split('"')[1]
    except FileNotFoundError:
        return ''

In [ ]:
#| hide
#|eval: false
_win_find_path()

'C:\\Program Files\\Stata18\\StataMP-64.exe'

In [ ]:
#| export
def _mac_find_path(_dir=None):
    """
    Attempt to find Stata path on macOS when not on user's PATH.
    Modified from stata_kernel's original to "/Applications/StataNow" and "/Applications/Stata".

    Returns:
        (str): Path to Stata. Empty string if not found.
    """
    if _dir is None:
        dirs = [r'/Applications/StataNow',
                r'/Applications/Stata']
    else:
        dirs = [_dir]    
    for this_dir in dirs:
        path = Path(this_dir)
        if os.path.exists(path):
            try:
                # find the application with the suffix .app
                # example path: /Applications/Stata/StataMP.app
                return str(next(path.glob("Stata*.app")))
            except StopIteration:
                return ''

In [ ]:
#| hide
_mac_find_path()

''

In [ ]:
#| export
def _other_find_path():
    for i in ['stata-mp', 'stata-se', 'stata']:
        stata_path = which(i)
        if stata_path:
            return stata_path
    return ''

In [ ]:
#| hide
_other_find_path()

''

In [ ]:
#| export
def _find_path(_dir=None):
    if os.getenv('CONTINUOUS_INTEGRATION'):
        print('WARNING: Running as CI; Stata path not set correctly')
        return 'stata'
    path = ''
    if platform.system() == 'Windows':
        path = _win_find_path(_dir)
    elif platform.system() == 'Darwin':
        path = _mac_find_path(_dir)
    return path if path else _other_find_path()

In [ ]:
#| hide
#|eval: false
_find_path()

'C:\\Program Files\\Stata18\\StataMP-64.exe'

In [ ]:
#| export
def _edition(stata_exe):
    edition = 'be'
    for e in ('be', 'se', 'mp'):
        if stata_exe.find(e) > -1:
            edition = e
            break
    return edition

In [ ]:
#| hide
test_eq(_edition(os.path.basename("/Applications/Stata/StataMP.app").lower()), "mp")
test_eq(_edition('StataMP-64.exe'.lower()), "mp")
test_eq(_edition(''), "be")
test_eq(_edition('...be...mp'), "be")

In [ ]:
#| export
def find_dir_edition(stata_path=None):
    if stata_path is None:
        stata_path = _find_path()
    if not stata_path:
        raise OSError("Stata path not found.")
    stata_dir = str(os.path.dirname(stata_path))
    stata_exe = str(os.path.basename(stata_path)).lower()
    return stata_dir, _edition(stata_exe)

In [ ]:
test_eq(find_dir_edition('C:/Program Files/Stata17/StataMP-64.exe'), ('C:/Program Files/Stata17', "mp"))
with ExceptionExpected(OSError):
    find_dir_edition('')

In [ ]:
#| export
def find_edition(stata_dir):
    stata_path = _find_path(stata_dir)
    stata_exe = str(os.path.basename(stata_path)).lower()
    return _edition(stata_exe)

In [ ]:
if _find_path('') == '': test_eq(find_edition(''), "be")
find_edition('C:\\Program Files\\Stata18')

'mp'

In [ ]:
from nbstata.misc_utils import Timer

In [ ]:
#|eval: false
with Timer():
    print(find_dir_edition())

('C:\\Program Files\\Stata18', 'mp')
Elapsed time: 0.0004 seconds


### Initialize pystata

In [ ]:
#| export
def set_pystata_path(stata_dir=None):
    stata_dir = stata_dir.strip('"\'')
    if stata_dir is None:
        stata_dir, _ = find_dir_edition()
    if not os.path.isdir(stata_dir):
        raise OSError(f'Specified stata_dir, {stata_dir}, is not a valid directory path')
    if not os.path.isdir(os.path.join(stata_dir, 'utilities')):
        raise OSError(f'Specified stata_dir, {stata_dir}, is not Stata\'s installation path')
    sys.path.append(os.path.join(stata_dir, 'utilities'))

In [ ]:
#|eval: false
with Timer():
    set_pystata_path()
    import pystata

Elapsed time: 0.0030 seconds


In [ ]:
#|eval: false
#| hide
set_pystata_path('C:\\Program Files\\Stata18')

In [ ]:
#|eval: false
with ExceptionExpected(): import sfi

In [ ]:
#| export
def launch_stata(stata_dir=None, edition=None, splash=True):
    """
    We modify stata_setup to make splash screen optional
    """
    if stata_dir is None:
        stata_dir, edition_found = find_dir_edition()
        edition = edition_found if edition is None else edition
    elif edition is None:
        edition = find_edition(stata_dir)
    set_pystata_path(stata_dir)
    import pystata
    try:
        if version.parse(pystata.__version__) >= version.parse("0.1.1"):
            # Splash message control is a new feature of pystata-0.1.1
            pystata.config.init(edition, splash=splash)
        else:
            pystata.config.init(edition)
    except FileNotFoundError as err:
        raise OSError(f'Specified edition, "{edition}", is not present at "{stata_dir}"')

In [ ]:
#|eval: false
with Timer():
    launch_stata(splash=False)
    pystata.config.status()

    System information
      Python version         3.11.10
      Stata version          Stata 18.0 (MP)
      Stata library path     C:\Program Files\Stata18\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = default, height = default
      graphic format         svg
Elapsed time: 1.9647 seconds


`sfi` can only be imported after Stata is launched:

In [ ]:
#|eval: false
import sfi

### Configure pystata graph output

https://www.stata.com/python/pystata18/config.html#pystata.config.set_graph_format

In [ ]:
#| export
def set_graph_format(gformat):
    import pystata
    if gformat == 'pystata':
        gformat = 'svg' # pystata default
    pystata.config.set_graph_format(gformat)

In [ ]:
#| eval: False
with Timer():
    set_graph_format('png')
pystata.config.status()

Elapsed time: 0.0000 seconds
    System information
      Python version         3.11.10
      Stata version          Stata 18.0 (MP)
      Stata library path     C:\Program Files\Stata18\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = default, height = default
      graphic format         png


In [ ]:
#| export
def _set_graph_size(width, height):
    import pystata
    pystata.config.set_graph_size(width, height)

In [ ]:
#| eval: False
_set_graph_size('2in', '4')
pystata.config.status()
_set_graph_size('default', 'default')

    System information
      Python version         3.11.10
      Stata version          Stata 18.0 (MP)
      Stata library path     C:\Program Files\Stata18\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = 2.0in, height = 4in
      graphic format         png


## nbstata configuration

The `Config` class handles the [configuration file](https://hugetim.github.io/nbstata/user_guide.html#configuration-optional), after which pystata may be initialized with `Config.init_stata`. `Config` then also handles configuration changes made by the [%set magic](https://hugetim.github.io/nbstata/user_guide.html#set-set) (that is, `StataMagics.magic_set`).

In [ ]:
#| export
def _get_config_settings(cpath):
    parser = configparser.ConfigParser(
        empty_lines_in_values=False,
        comment_prefixes=('*','//'),
        inline_comment_prefixes=('//',),
    )
    parser.read(str(cpath))
    return dict(parser.items('nbstata'))

In [ ]:
#| export
def xdg_user_config_path():
    xdg_config_home = Path(os.environ.get('XDG_CONFIG_HOME', Path.home() / '.config'))
    return xdg_config_home / 'nbstata/nbstata.conf'

def old_user_config_path():
    return Path('~/.nbstata.conf').expanduser()

In [ ]:
xdg_user_config_path()

Path('C:/Users/tjhuegerich/.config/nbstata/nbstata.conf')

In [ ]:
old_user_config_path()

Path('C:/Users/tjhuegerich/.nbstata.conf')

In [ ]:
#| export
class Config:
    "nbstata configuration"
    env = {'stata_dir': None,
           'edition': None,
           'splash': 'False',
           'graph_format': 'png',
           'graph_width': '5.5in',
           'graph_height': '4in',
           'echo': 'None',
           'missing': '.',
           'browse_auto_height': 'True',
          }
    valid_values_of = dict(
        edition={None, 'mp', 'se', 'be'},
        graph_format={'pystata', 'svg', 'png', 'pdf'},
        echo={'True', 'False', 'None'},
        splash={'True', 'False'},
        browse_auto_height={'True', 'False'},
    )
    
    @property
    def splash(self):
        return False if self.env['splash'] == 'False' else True
    
    @property
    def browse_auto_height(self):
        return False if self.env['browse_auto_height'] == 'False' else True
      
    @property
    def noecho(self):
        return self.env['echo'] == 'None'
    
    @property
    def echo(self):
        return self.env['echo'] == 'True'
    
    def display_status(self):
        import pystata
        pystata.config.status()
        print(f"""
      echo                   {self.env['echo']}
      missing                {self.env['missing']}
      browse_auto_height     {self.env['browse_auto_height']}
      config file path       {self.config_path}""")

    def __init__(self):
        """First check if a configuration file exists. If not, try `find_dir_edition`."""
        self.errors = []
        self._update_backup_graph_size()
        self.config_path = None

    def _update_backup_graph_size(self):
        self.backup_graph_size = {key: self.env[key] for key in {'graph_width', 'graph_height'}}
                
    def process_config_file(self):
        global_config_path = Path(os.path.join(sys.prefix, 'etc', 'nbstata.conf'))
        for cpath in (xdg_user_config_path(), old_user_config_path(), global_config_path):      
            if cpath.is_file():
                self._get_config_env(cpath)
                break
            
    def _get_config_env(self, cpath):
        try:
            settings = _get_config_settings(cpath)
        except configparser.Error as err:
            print_red(f"Configuration error in {cpath}:\n"
                      f"    {str(err)}")
        else:
            self.config_path = str(cpath)
            self.update(
                settings, 
                init=True, 
                error_header=f"Configuration errors in {self.config_path}:"  
            )
            
    def update(self, env, init=False, error_header="%set error(s):"):
        init_only_settings = {'stata_dir','edition','splash'}
        allowed_settings = self.env if init else set(self.env)-init_only_settings
        for key in list(env):
            if key not in allowed_settings:
                explanation = (
                    "is only allowed in a configuration file." if key in init_only_settings
                    else "is not a valid setting."
                )
                self.errors.append(f"    '{key}' {explanation}")
                env.pop(key)
            elif key in self.valid_values_of and env[key] not in self.valid_values_of[key]:
                self.errors.append(
                    f"    '{key}' configuration invalid: '{env[key]}' is not a valid value. "
                    f"Reverting to: {key} = {self.env[key]}"
                )
                env.pop(key)
        self._display_and_clear_update_errors(error_header)
        for key in set(env)-{'graph_width', 'graph_height'}:
            if not init: print(f"{key} was {self.env[key]}, is now {env[key]}")
        self.env.update(env)
  
    def _display_and_clear_update_errors(self, error_header):
        if self.errors:
            print_red(error_header)
        for message in self.errors:
            print_red(message)
        self.errors = []

The below example reads in from a sample configuration file:

In [ ]:
config = Config()
config.process_config_file()
config.env

{'stata_dir': 'C:\\Program Files\\Stata18',
 'edition': 'mp',
 'splash': 'False',
 'graph_format': 'png',
 'graph_width': 'default',
 'graph_height': 'default',
 'echo': 'None',
 'missing': '.',
 'browse_auto_height': 'True'}

Testing out error messages explaining invalid keys:

In [ ]:
#| eval: False
config.update({'splash': 'True'})

%set error(s):
    'splash' is only allowed in a configuration file.


In [ ]:
#| eval: False
config.update({'splash': 'True'}, init=True)

In [ ]:
#| eval: False
config.update({'not_a_key': 'True'})

%set error(s):
    'not_a_key' is not a valid setting.


The configuration file is read in prior to loading Stata (since it can contain a path to the desired Stata executable). But checking the validity of graph size configuration settings uses Stata, so that can't be done in the same step in which the configuration file is read in. Thus, the following workaround is used: hold the read-in graph size settings until they are actually applied, reverting to previous valid settings if they don't work:

In [ ]:
#| export
@patch_to(Config)
def set_graph_size(self, init=False):
    try:
        _set_graph_size(self.env['graph_width'], self.env['graph_height'])
    except ValueError as err:
        self.env.update(self.backup_graph_size)
        print_red(f"Configuration error: {str(err)}. Graph size not changed.")
        if init: self.set_graph_size() # ensures set to definite measures rather than "default"
    else:
        if {key: self.env[key] for key in {'graph_width', 'graph_height'}} != self.backup_graph_size:
            if not init:
                print(f"graph size was ({self.backup_graph_size['graph_width']}, "
                      f"{self.backup_graph_size['graph_height']}), "
                      f"is now ({self.env['graph_width']}, {self.env['graph_height']}).")
            self._update_backup_graph_size()

If the configuration file has invalid width/height, the error message says "Graph size not changed" even though, under the hood, the *pystata* graph size configuration is changing from "default" to definite measures. This behavior ensures that using the %set magic to change just one of the size values, width or height, always exhibits the behavior described in the *nbstata* [user guide](https://hugetim.github.io/nbstata/user_guide.html#configuration-optional) rather than the (maintained aspect ratio) [behavior described in the pystata docs](https://www.stata.com/python/pystata18/config.html#pystata.config.set_graph_size). 

In [ ]:
#| eval: False
config.display_status()
config.set_graph_size(init=True)
config.display_status()

    System information
      Python version         3.11.10
      Stata version          Stata 18.0 (MP)
      Stata library path     C:\Program Files\Stata18\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = default, height = default
      graphic format         png

      echo                   None
      missing                .
      browse_auto_height     True
      config file path       C:\Users\tjhuegerich\.config\nbstata\nbstata.conf
    System information
      Python version         3.11.10
      Stata version          Stata 18.0 (MP)
      Stata library path     C:\Program Files\Stata18\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = default, height = default
      graphic format         png

      echo                   None
      missing           

In [ ]:
#| eval: False
config.update({'graph_width': '3'})
config.set_graph_size()

graph size was (default, default), is now (3, default).


In [ ]:
#| eval: False
config.display_status()
config.env

    System information
      Python version         3.11.10
      Stata version          Stata 18.0 (MP)
      Stata library path     C:\Program Files\Stata18\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = 3in, height = default
      graphic format         png

      echo                   None
      missing                .
      browse_auto_height     True
      config file path       C:\Users\tjhuegerich\.config\nbstata\nbstata.conf


{'stata_dir': 'C:\\Program Files\\Stata18',
 'edition': 'mp',
 'splash': 'True',
 'graph_format': 'png',
 'graph_width': '3',
 'graph_height': 'default',
 'echo': 'None',
 'missing': '.',
 'browse_auto_height': 'True'}

In [ ]:
#| eval: False
config.update({'graph_height': '-3'})
config.set_graph_size()
config.env

Configuration error: graph height is invalid. Graph size not changed.


{'stata_dir': 'C:\\Program Files\\Stata18',
 'edition': 'mp',
 'splash': 'True',
 'graph_format': 'png',
 'graph_width': '3',
 'graph_height': 'default',
 'echo': 'None',
 'missing': '.',
 'browse_auto_height': 'True'}

In [ ]:
#| export
@patch_to(Config)
def update_graph_config(self, init=False):
    graph_format = self.env['graph_format']
    if graph_format == 'pystata':
        graph_format = 'svg'
    set_graph_format(graph_format)
    self.set_graph_size(init)

In [ ]:
#| eval: False
config.update_graph_config()

In [ ]:
#| export
@patch_to(Config)
def init_stata(self):
    launch_stata(self.env['stata_dir'],
                 self.env['edition'],
                 self.splash,
                )
    self.update_graph_config(init=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()